In [339]:
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [340]:
import requests
from bs4 import BeautifulSoup


class BaseLoader:
    def __init__(self, urls):
        self.urls = urls

    def load(self):
        references = []
        for url in self.urls:
            response = requests.get(url, verify=False)
            soup = BeautifulSoup(response.text, 'html.parser')
            references.append(soup.get_text())
        
        return references


In [341]:
with open("hypos.txt") as f:
    hypos = [line.strip() for line in f]

with open("refs.txt") as f:
    refs = [line.strip() for line in f]

In [342]:
refs

['https://www.hotcourses.vn/study-abroad-info/study-guides/nganh-dien-tu-vien-thong/',
 'https://blog.jobsnew.vn/nganh-dien-tu-vien-thong/',
 'https://glints.com/vn/blog/co-hoi-viec-lam-nganh-dien-tu-vien-thong/',
 'https://ts.hust.edu.vn/training-cate/nganh-dao-tao-dai-hoc/dien-tu-va-vien-thong',
 'https://ictvietnam.vn/4-xu-huong-dinh-hinh-tuong-lai-cua-nganh-vien-thong-nam-2023-55407.html',
 'https://baonghean.vn/5-xu-huong-noi-bat-se-dinh-hinh-linh-vuc-vien-thong-vao-nam-2024-post281547.html']

In [343]:
loader = BaseLoader(refs)
refs= loader.load()

In [344]:
len(refs)

6

In [345]:
refs

['\n\n\nNgành Điện tử - Viễn Thông: Học gì, học ở đâu và cơ hội nghề nghiệp\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nTới nội dung chính\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nKhóa học & Học bổng\n\n\n\n\n\nTìm học bổng 2024 \n\nTất cả học bổng du học\nHọc bổng du học thạc sĩ\nHọc bổng bậc cử nhân\nHọc bổng học nghề\nHọc bổng thành tích\n\n\n\nTìm khóa học\n\n\nChương trình cử nhân\nChương trình sau đại học\nHọc nghề\nDự bị đại học\n\nDự bị thạc sĩ\nKhóa học tiếng Anh\nChương trình MBA\n\n\n\n\nCác ngành học phổ biến \n\n\nSức khỏe và Y học\n\nDu lịch và Quản trị Nhà hàng, Khách sạn\n\nGiáo đục và Đào tạo\n\nKhoa học Máy tính và CNTT\n\nKhoa học Ứng dụng và Khoa học cơ bản\n\n\n\nNghiên cứu Xã hội và Truyền thông\n\nKiến trúc và Xây dựng\n\nKinh doanh và Quản lý\n\nKỹ thuật\n\nNghệ thuật Sáng tạo và Thiết kế\n\n\n\n\n\n\n\n\n\n\nĐi

In [346]:
import re

def normalize_text(text):
    # Remove excess whitespace and newline characters
    text = re.sub(r'\s+', ' ', text)
    # Remove special characters and non-alphanumeric characters
    text = re.sub(r'[^\w\s]', '', text)
    # Convert to lowercase
    text = text.lower()
    return text

In [347]:
hypos = [' '.join(normalize_text(hypo) for hypo in hypos)]

In [348]:
hypos

[' cơ hội nghề nghiệp cho sinh viên ngành điện tử  viễn thông của đại học bách khoa hà nội sau khi tốt nghiệp   tổng quan ngành điện tử  viễn thông  ngành điện tử  viễn thông là một trong những ngành kỹ thuật chủ chốt đóng vai trò quan trọng trong sự phát triển của xã hội hiện đại sự tiến bộ không ngừng của công nghệ thông tin và viễn thông đã và đang tạo ra nhiều cơ hội nghề nghiệp mới đồng thời đòi hỏi nguồn nhân lực có trình độ chuyên môn cao và khả năng thích ứng với sự thay đổi nhanh chóng của công nghệ hotcoursesvn   cơ hội nghề nghiệp sau khi tốt nghiệp  sinh viên tốt nghiệp ngành điện tử  viễn thông từ đại học bách khoa hà nội có thể đảm nhận nhiều vị trí công việc khác nhau trong lĩnh vực này các vị trí bao gồm chuyên viên tư vấn thiết kế và vận hành kỹ thuật tại các đài phát thanh đài truyền hình công ty tư vấn thiết kế mạng viễn thông sản xuất thiết bị điện tử  viễn thông và công ty sản xuất vi mạch hotcoursesvn  ngoài ra sinh viên cũng có thể làm việc tại các công ty viễn t

In [349]:
refs = [normalize_text(ref) for ref in refs]

In [350]:
refs

[' ngành điện tử  viễn thông học gì học ở đâu và cơ hội nghề nghiệp tới nội dung chính khóa học  học bổng tìm học bổng 2024 tất cả học bổng du học học bổng du học thạc sĩ học bổng bậc cử nhân học bổng học nghề học bổng thành tích tìm khóa học chương trình cử nhân chương trình sau đại học học nghề dự bị đại học dự bị thạc sĩ khóa học tiếng anh chương trình mba các ngành học phổ biến sức khỏe và y học du lịch và quản trị nhà hàng khách sạn giáo đục và đào tạo khoa học máy tính và cntt khoa học ứng dụng và khoa học cơ bản nghiên cứu xã hội và truyền thông kiến trúc và xây dựng kinh doanh và quản lý kỹ thuật nghệ thuật sáng tạo và thiết kế điểm đến lên kế hoạch du học nào điểm đến du học nào phù hợp với bạn nhất tìm hiểu ngay canada các trường đại học ở canada du học canada các ngành học ở canada kinh nghiệm xin visa du học canada úc các trường đại học ở úc du học úc các ngành học ở úc kinh nghiệm xin visa du học úc vương quốc anh các trường đại học ở anh du học anh các ngành học ở anh kin

In [351]:
len(hypos), len(refs)

(1, 6)

### Using BERTScore for Evaluation

In [352]:
from bert_score import score

In [353]:
# Hide the loading messages
import logging
import transformers
transformers.tokenization_utils.logger.setLevel(logging.ERROR)
transformers.configuration_utils.logger.setLevel(logging.ERROR)
transformers.modeling_utils.logger.setLevel(logging.ERROR)

In [354]:
def bert_score(hypos, refs, lang="vi"):
    bert_scores = []
    for ref in refs:
        scores = score(hypos, [ref], lang=lang, verbose=False)
        bert_scores.append(scores)
    return bert_scores

In [355]:
bert_scores = bert_score(hypos, refs)

In [356]:
import torch

precision = torch.mean(torch.stack([bert_score[0] for bert_score in bert_scores]))
recall = torch.mean(torch.stack([bert_score[1] for bert_score in bert_scores]))
f1 = torch.mean(torch.stack([bert_score[2] for bert_score in bert_scores]))

In [357]:
print(f"Precision: {precision.item():.2f}")
print(f"Recall: {recall.item():.2f}")
print(f"F1: {f1.item():.2f}")

Precision: 0.64
Recall: 0.64
F1: 0.64


### Using ROUGE score for Evaluation

In [358]:
from rouge_score import rouge_scorer

def rouge_score(hypos, refs):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2'], use_stemmer=True)
    scores = [scorer.score(ref, hypo) for ref, hypo in zip(refs, hypos)]
    return scores

rouge = rouge_score(hypos, refs)

In [359]:
for i, score in enumerate(rouge):
    print("ROUGE scores")
    print(f"ROUGE-1: P = {score['rouge1'].precision:.2f}, R = {score['rouge1'].recall:.2f}, F1 = {score['rouge1'].fmeasure:.2f}")
    print(f"ROUGE-2: P = {score['rouge2'].precision:.2f}, R = {score['rouge2'].recall:.2f}, F1 = {score['rouge2'].fmeasure:.2f}")
    print()

ROUGE scores
ROUGE-1: P = 0.98, R = 0.26, F1 = 0.42
ROUGE-2: P = 0.83, R = 0.22, F1 = 0.35



### Review
1. Summarization is a complex task, even advanced models can struggle to accurately capture all the important information from a source text.

2. BERTScore or ROUGE that these metrics focus on things like including keywords from the source text, so that do not perfectly reflect human judgement of a good summary.

3. The response of chatbot from many different sources.


### HUMAN EVALUATION IS THE BEST WAY TO EVALUATE THE QUALITY OF A SUMMARY